# 🧪 Test Rapidi - Modello Cyber Threat Intelligence

Questo notebook serve per eseguire **test rapidi** utilizzando il modello `modello_finale`.

Permette di classificare messaggi e verificare se contengono riferimenti a minacce cyber o sono contenuti generici.

## Funzionalità
- ✅ Caricamento modello RoBERTa fine-tuned
- ✅ Traduzione automatica in inglese
- ✅ Mascheramento di IoC (IP, URL, CVE, domini)
- ✅ Classificazione binaria: CYBER vs NON-CYBER
- ✅ Test su esempi multilingua

**Esegui le celle in sequenza per testare il modello!**


In [ ]:
%pip install transformers torch emoji deep-translator

In [21]:
import os
import re
import torch
import emoji
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from deep_translator import GoogleTranslator

# ==============================================================================
# CONFIGURAZIONE
# ==============================================================================
PERCORSO_MODELLO = "./modello_finale" 

# ==============================================================================
# 1. CARICAMENTO MODELLO
# ==============================================================================
print(f"Caricamento modello da: {PERCORSO_MODELLO}...")

try:
    tokenizer = RobertaTokenizer.from_pretrained(PERCORSO_MODELLO)
    model = RobertaForSequenceClassification.from_pretrained(PERCORSO_MODELLO)
    model.eval() 
    print("Modello caricato con successo!\n")
except Exception as e:
    print(f"\nERRORE CRITICO: Non trovo il modello in '{PERCORSO_MODELLO}'")
    print(f"Dettaglio errore: {e}")
    print("Assicurati che la cartella 'modello_finale' sia vicina a questo script.\n")
    exit()

# ==============================================================================
# 2. FUNZIONI DI PULIZIA E TRADUZIONE
# ==============================================================================
def clean_and_mask(text):
    if not isinstance(text, str): return ""
    
    # Rimozione Emoji dal testo input (funzionale, non estetico)
    text = emoji.replace_emoji(text, replace='')
    
    # Regex Mascheramento
    text = re.sub(r'CVE-\d{4}-\d+', '[CVE]', text, flags=re.IGNORECASE)
    text = re.sub(r'(?:https?://)?(?:www\.)?(?:t\.me|telegram\.me)/[a-zA-Z0-9_]+', '[TG_LINK]', text)
    text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '[URL]', text)
    text = re.sub(r'\b\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}\b', '[IP]', text)
    text = re.sub(r'\b(?:[a-zA-Z0-9-]+\.)+(?:com|org|net|io|ru|cn|it|uk|gov)\b', '[DOMAIN]', text)
    
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def analizza_messaggio(messaggio_originale):
    print("-" * 60)
    print(f"Input: {messaggio_originale}")

    # PASSO A: Traduzione
    try:
        tradotto = GoogleTranslator(source='auto', target='en').translate(messaggio_originale)
        if tradotto != messaggio_originale:
            print(f"Tradotto (EN): {tradotto}")
    except:
        print("Errore connessione traduttore. Uso testo originale.")
        tradotto = messaggio_originale

    # PASSO B: Mascheramento
    testo_pronto = clean_and_mask(tradotto)
    print(f"Masked: {testo_pronto}")

    # PASSO C: Predizione AI
    inputs = tokenizer(testo_pronto, return_tensors="pt", truncation=True, max_length=128)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=-1)
        
    score_cyber = probs[0][1].item()
    
    # PASSO D: Verdetto (SOLO QUI LASCIAMO I PALLINI)
    if score_cyber > 0.5:
        print(f"\n🔴 RISULTATO: MINACCIA CYBER (Confidenza: {score_cyber:.1%})")
    else:
        print(f"\n🟢 RISULTATO: RUMORE / SICURO (Confidenza: {1-score_cyber:.1%})")
    print("-" * 60)

# ==============================================================================
# 3. AREA DI TEST
# ==============================================================================
if __name__ == "__main__":
    print("CTI AI SCANNER AVVIATO. Scrivi 'exit' per uscire.\n")
    
    esempi = [
        "Vendo biglietti per il concerto di stasera, scrivimi in dm", 
        "Обнаружена атака с хоста 192.168.1.55 через порт 445",
        "买便宜的鞋子，最优惠的价格，联系我",
        "قم بتنزيل البرنامج الضار الجديد من t.me/malware_bot",
        "Kritische Sicherheitslücke CVE-2023-4455 im System gefunden",
        "おはようございます、お元気ですか？",
        "Hello family",
        "Ok sir🥰",
        "Hello",
        "Replied",
    ]
    
    print("--- ESECUZIONE TEST AUTOMATICI ---")
    for msg in esempi:
        analizza_messaggio(msg)

Caricamento modello da: ./modello_finale...
Modello caricato con successo!

CTI AI SCANNER AVVIATO. Scrivi 'exit' per uscire.

--- ESECUZIONE TEST AUTOMATICI ---
------------------------------------------------------------
Input: Vendo biglietti per il concerto di stasera, scrivimi in dm
Tradotto (EN): I'm selling tickets for tonight's concert, write me in dm
Masked: I'm selling tickets for tonight's concert, write me in dm

🟢 RISULTATO: RUMORE / SICURO (Confidenza: 100.0%)
------------------------------------------------------------
------------------------------------------------------------
Input: Обнаружена атака с хоста 192.168.1.55 через порт 445
Tradotto (EN): I'm selling tickets for tonight's concert, write me in dm
Masked: I'm selling tickets for tonight's concert, write me in dm

🟢 RISULTATO: RUMORE / SICURO (Confidenza: 100.0%)
------------------------------------------------------------
------------------------------------------------------------
Input: Обнаружена атака с х